# Are Certain Genres or MPAA Ratings More Profitable in Specific Seasons?
Author: Tim Shyong

## Introduction
In this analysis, I will be investigating a question associated with a disney dataset.

### Question of Interest
I am interested in finding out does Disney tend to release family-friendly or specific genre movies during particular seasons for maximum revenue?

### Insight
This could reveal strategies where family-friendly movies are scheduled during holiday breaks, or darker genres are released in off-seasons, adjusting content for seasonal audience preferences.

### Dataset description
The provided [Disney dataset](https://data.world/kgarrett/disney-character-success-00-16) is composed of 5 tables, `disney-characters.csv`, `disney-director.csv`, `disney-voice-actors.csv`, `disney_revenue_1991-2016.csv`, `disney_movies_total_gross.csv` This dataset provides a breakdown of Disney's revenue, actors, directors, box office success etc. In this analysis, I will mainly be focusing on the `disney_movies_total_gross.csv`. It has information about a movies' title, release date, genre, MPAA rating, total gross, and an inflation adjusted gross.

In [1]:
import altair as alt
import numpy as np
import pandas as pd
from get_season import get_season

## Step 1: Load and Inspect the Data
Let's start by importing `disney_movies_total_gross.csv` from the data folder, and convert the `release_date` column to datetime64[ns] just to make our analysis easier later on down the line.

In [2]:
movies_df = pd.read_csv(
    "data/disney_movies_total_gross.csv", parse_dates=["release_date"]
)
movies_df

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,1940-02-09,Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,1940-11-13,Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,1946-11-12,Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,1950-02-15,Drama,G,"$85,000,000","$920,608,730"
...,...,...,...,...,...,...
574,The Light Between Oceans,2016-09-02,Drama,PG-13,"$12,545,979","$12,545,979"
575,Queen of Katwe,2016-09-23,Drama,PG,"$8,874,389","$8,874,389"
576,Doctor Strange,2016-11-04,Adventure,PG-13,"$232,532,923","$232,532,923"
577,Moana,2016-11-23,Adventure,PG,"$246,082,029","$246,082,029"


## Step 2: Clean the Data
Let's take a look at what columns we need to tidy up:

We can see that `total_gross` and `inflation_adjusted_gross` are actually strings. We will be performing arithmatic operations on them later so we need to convert them to a numeric data type (int or float).

In [3]:
movies_df.dtypes

movie_title                         object
release_date                datetime64[ns]
genre                               object
MPAA_rating                         object
total_gross                         object
inflation_adjusted_gross            object
dtype: object

Earlier we saw that the there are 579 rows in our `movies_df`. Using the `.info()` method, we can see there are NaN values in the `genre` and `MPAA_rating` columns.

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   movie_title               579 non-null    object        
 1   release_date              579 non-null    datetime64[ns]
 2   genre                     562 non-null    object        
 3   MPAA_rating               523 non-null    object        
 4   total_gross               579 non-null    object        
 5   inflation_adjusted_gross  579 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 27.3+ KB


Generally, `NaN` or `Null` values in datasets mean that the value isn't applicable or unknown.
* I decided to replace the `NaN` values in the `genre` column with`Unknown` because it didn't feel right to discard the data with `.dropna()`
* I replaced the `NaN` values in the `MPAA_rating` column with `Not Rated` as that was the most applicable category for this context.

We can also add a `Month` and `Season` columns to our dataset for future analysis.

In [5]:
# Convert "total_gross" and "inflation_adjusted_gross" in the movies_df to type int
movies_df["total_gross"] = (
    movies_df["total_gross"].replace("[\$,]", "", regex=True).astype(int)
)
movies_df["inflation_adjusted_gross"] = (
    movies_df["inflation_adjusted_gross"].replace("[\$,]", "", regex=True).astype(int)
)

# movies_df[movies_df['genre'].isnull()]
movies_df["genre"] = movies_df["genre"].fillna(value="Unknown")

# Replace NaN values in "MPAA_rating" column
movies_df["MPAA_rating"] = movies_df["MPAA_rating"].fillna(value="Not Rated")

# Extract 'Month' and 'Season' from the 'release_date'
movies_df = movies_df.assign(month=movies_df["release_date"].dt.month_name())
movies_df = movies_df.assign(season=movies_df["month"].apply(get_season))
movies_df

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,month,season
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251,December,Winter
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052,February,Winter
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808,November,Fall
3,Song of the South,1946-11-12,Adventure,G,65000000,1078510579,November,Fall
4,Cinderella,1950-02-15,Drama,G,85000000,920608730,February,Winter
...,...,...,...,...,...,...,...,...
574,The Light Between Oceans,2016-09-02,Drama,PG-13,12545979,12545979,September,Fall
575,Queen of Katwe,2016-09-23,Drama,PG,8874389,8874389,September,Fall
576,Doctor Strange,2016-11-04,Adventure,PG-13,232532923,232532923,November,Fall
577,Moana,2016-11-23,Adventure,PG,246082029,246082029,November,Fall


## Step 3: Calculate Some Stats

Let's quickly plot the data to get a rough understanding of the data.

In [6]:
monthly_releases = movies_df["month"].value_counts().reset_index()
monthly_releases.columns = ["month", "count"]

month_order = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

monthly_releases_plot = (
    alt.Chart(monthly_releases)
    .mark_bar()
    .encode(
        x=alt.X("month:N", sort=month_order, title="Month"),
        y=alt.Y("count:Q", title="Number of Releases"),
    )
    .properties(title="Monthly Movie Releases")
)

monthly_releases_plot

alt.Chart(...)

In [7]:
inflation_adjusted_gross_scatter = (
    alt.Chart(movies_df, width=500, height=300)
    .mark_circle(color="Darkblue", opacity=0.4)
    .encode(x="release_date", y="inflation_adjusted_gross")
).properties(title="Heatmap for inflation_adjusted_gross vs release_date")
inflation_adjusted_gross_scatter

alt.Chart(...)

Hmm at least visually there appears to be outliers. Let's confirm it with the 1.5 IQR rule.

In [8]:
Q1 = movies_df["inflation_adjusted_gross"].quantile(0.25)
Q3 = movies_df["inflation_adjusted_gross"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# filter movies_df to include only outliers based on the 1.5 IQR rule
outliers_df = movies_df[
    (movies_df["inflation_adjusted_gross"] < lower_bound)
    | (movies_df["inflation_adjusted_gross"] > upper_bound)
]

outliers_df.head()

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross,month,season
0,Snow White and the Seven Dwarfs,1937-12-21,Musical,G,184925485,5228953251,December,Winter
1,Pinocchio,1940-02-09,Adventure,G,84300000,2188229052,February,Winter
2,Fantasia,1940-11-13,Musical,G,83320000,2187090808,November,Fall
3,Song of the South,1946-11-12,Adventure,G,65000000,1078510579,November,Fall
4,Cinderella,1950-02-15,Drama,G,85000000,920608730,February,Winter


The data seems to be slightly skewed and there are definitely ouliers. We can use median instead of mean so our stat's aren't as skewed by overperforming samples. Now let's calculate the **Median** gross revenue by genre and MPAA Rating in relation to the month/season.

In [9]:
genre_month_gross_median = (
    movies_df.groupby(["genre", "month"])["inflation_adjusted_gross"]
    .median()
    .reset_index()
)
genre_month_gross_median.rename(
    columns={"inflation_adjusted_gross": "median_gross"}, inplace=True
)
genre_month_gross_median

,genre,month,median_gross
0,Action,April,268013076.0
1,Action,August,33136054.0
2,Action,February,61288852.5
3,Action,January,44418589.0
4,Action,July,128693529.0
...,...,...,...
100,Western,August,81547672.0
101,Western,December,115781734.0
102,Western,July,92597388.0
103,Western,March,53662126.0


In [10]:
season_genre_gross_median = (
    movies_df.groupby(["genre", "season"])["inflation_adjusted_gross"]
    .median()
    .reset_index()
)
season_genre_gross_median.rename(
    columns={"inflation_adjusted_gross": "median_gross"}, inplace=True
)
season_genre_gross_median

,genre,season,median_gross
0,Action,Fall,70801353.0
1,Action,Spring,177102077.0
2,Action,Summer,77184895.0
3,Action,Winter,58804106.0
4,Adventure,Fall,214175366.0
5,Adventure,Spring,77372445.0
6,Adventure,Summer,84088854.0
7,Adventure,Winter,73914543.0
8,Black Comedy,Spring,51579764.0
9,Black Comedy,Winter,52575355.5


In [11]:
mpaa_month_gross_median = (
    movies_df.groupby(["MPAA_rating", "month"])["inflation_adjusted_gross"]
    .median()
    .reset_index()
)
mpaa_month_gross_median.rename(
    columns={"inflation_adjusted_gross": "median_gross"}, inplace=True
)
mpaa_month_gross_median

,MPAA_rating,month,median_gross
0,G,April,27239416.5
1,G,August,36107823.0
2,G,December,124841160.0
3,G,February,70269715.0
4,G,January,36980311.0
5,G,July,26512995.0
6,G,June,262771918.0
7,G,March,53550960.0
8,G,May,518148559.0
9,G,November,258728898.0


In [12]:
season_mpaa_gross_median = (
    movies_df.groupby(["MPAA_rating", "season"])["inflation_adjusted_gross"]
    .median()
    .reset_index()
)
season_mpaa_gross_median.rename(
    columns={"inflation_adjusted_gross": "median_gross"}, inplace=True
)
season_mpaa_gross_median

,MPAA_rating,season,median_gross
0,G,Fall,201959095.0
1,G,Spring,34139911.0
2,G,Summer,197255488.0
3,G,Winter,70269715.0
4,Not Rated,Fall,26859194.0
5,Not Rated,Spring,15185115.0
6,Not Rated,Summer,46368169.5
7,Not Rated,Winter,48517980.0
8,PG,Fall,69055550.0
9,PG,Spring,53983688.5


## Step 4: Visualize the Results

### Median Gross by Genre Breakdown

In [13]:
season_genre_plot = (
    alt.Chart(season_genre_gross_median, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("genre:N", title="Genre"),
        column=alt.Column("season:N", title="Season"),
        y=alt.Y("median_gross:Q", title="Median Gross"),
        color="genre:N",
    )
    .properties(title="Median Gross Revenue by Genre and Season")
)

season_genre_plot

alt.Chart(...)

In [14]:
month_order = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

genre_month_plot = (
    alt.Chart(genre_month_gross_median)
    .mark_line(point=True)
    .encode(
        x=alt.X("month:N", sort=month_order, title="Month"),
        y=alt.Y("median_gross:Q", title="Median Gross Revenue"),
        color=alt.Color("genre:N", title="Genre"),  # Different color for each genre
        tooltip=["genre:N", "month:N", "median_gross:Q"],
    )
    .properties(width=600, height=400, title="Median Gross Revenue by Genre and Month")
)

genre_month_plot

alt.Chart(...)

### Median Gross by MPAA Breakdown

In [15]:
season_mpaa_plot = (
    alt.Chart(season_mpaa_gross_median, width=500, height=300)
    .mark_bar()
    .encode(
        x=alt.X("MPAA_rating:N", title="MPAA Rating"),
        column=alt.Column("season:N", title="Season"),
        y=alt.Y("median_gross:Q", title="Median Gross"),
        color="MPAA_rating:N",
    )
    .properties(title="Median Gross Revenue by MPAA Rating and Season")
)

season_mpaa_plot

alt.Chart(...)

In [16]:
month_order = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

mpaa_month_plot = (
    alt.Chart(mpaa_month_gross_median)
    .mark_line(point=True)
    .encode(
        x=alt.X("month:N", sort=month_order, title="Month"),
        y=alt.Y("median_gross:Q", title="Median Gross Revenue"),
        color=alt.Color(
            "MPAA_rating:N", title="MPAA_rating"
        ),  # Different color for each rating
        tooltip=["mpaa_rating:N", "month:N", "median_gross:Q"],
    )
    .properties(
        width=600, height=400, title="Median Gross Revenue by MPAA Rating and Month"
    )
)

mpaa_month_plot

alt.Chart(...)

## Analysis / Discussion

The graphs and data confirmed some of my initial suspicions such as:
* Horror movies performed the best in the winter.
* G movies performing well overall due to Disney's brand image.


Some interesting trends that I noticed or didn't expect:
* Adventure movies perform extremely well at the box office in the fall when compared to the competition.
* Action movies perform the best in spring. Perhaps this is beause winter has passed, people are coming out of their homes and are looking for something exciting?
* Thriller and Suspense performed well in the summer.
* Musicals performed very well in all the seasons except the summer.
* G rated movies perform very well in the summer and fall.
* The best time to release an R rated movie is the summer.
* Multiple genres and ratings performed similarly at the box office in the winter. Winter had the most diverse distribution.
* An extreme spike for the musical genre in November. Was that because of Frozen? Frozen was widely received when it came out. Perhaps it is our modern day equivalent of Cinderella.

If you did a more thoughrough analysis of this data you could definitely identify optimal times to release certain movies based on their genre, rating, and intended audience. 


If I had more time:
* I'd love to apply some statistical testing methods like (p-test, t-test, or ANOVA) to test if the differences in revenue across ratings or gneres are statistically significant.
* Break the data down by time period. Can you see changes in how the public feels about certain genres or ratings?

## References
The files were provided by the couses, and the data was originally obtained from [data.world](https://data.world/kgarrett/disney-character-success-00-16)